In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import os
import logging
from pathlib import Path

# Ensure logs directory exists
os.makedirs('logs', exist_ok=True)

# Configure logging
logging.basicConfig(
    filename='logs/app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def extract_frames(video_path, output_dir, frames_per_second=1):
    """
    Extract one frame per second from a video and save to output_dir.
    
    Args:
        video_path (str): Path to the input video file.
        output_dir (str): Directory to save extracted frames.
        frames_per_second (int): Number of frames to extract per second (default: 1).
    
    Returns:
        list: List of saved frame paths.
    """
    try:
        # Ensure output directory exists
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # Open video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logging.error(f"Failed to open video: {video_path}")
            return []
        
        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps / frames_per_second)  # Frames to skip for 1 frame/sec
        frame_count = 0
        saved_frames = []
        
        # Extract video ID from filename
        video_id = Path(video_path).stem
        
        logging.info(f"Processing video: {video_path}, FPS: {fps}")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            if frame_count % frame_interval == 0:
                # Generate frame filename (e.g., frame_0001.jpg)
                frame_filename = f"frame_{frame_count:04d}.jpg"
                frame_path = os.path.join(output_dir, frame_filename)
                
                # Save frame as JPEG
                cv2.imwrite(frame_path, frame)
                saved_frames.append(frame_path)
                logging.info(f"Saved frame: {frame_path}")
            
            frame_count += 1
        
        # Release video capture
        cap.release()
        logging.info(f"Extracted {len(saved_frames)} frames from {video_path}")
        return saved_frames
    
    except Exception as e:
        logging.error(f"Error processing {video_path}: {str(e)}")
        return []

def main():
    # Input and output directories
    video_dir = "data/videos"
    frames_base_dir = "frames"
    
    # Ensure video directory exists
    if not os.path.exists(video_dir):
        logging.error(f"Video directory {video_dir} does not exist")
        return
    
    # Process each video in the directory
    for video_file in os.listdir(video_dir):
        if video_file.endswith(('.mp4', '.avi', '.mov')):  # Supported video formats
            video_path = os.path.join(video_dir, video_file)
            video_id = Path(video_file).stem
            output_dir = os.path.join(frames_base_dir, video_id)
            
            logging.info(f"Starting frame extraction for {video_file}")
            saved_frames = extract_frames(video_path, output_dir)
            logging.info(f"Completed extraction for {video_file}: {len(saved_frames)} frames")

if __name__ == "__main__":
    main()